# The nested logit model

## Specification of the utility functions

In [1]:
import pandas as pd
import biogeme.biogeme as bio
import biogeme.database as db
import biogeme.models as models
import biogeme.optimization as opt
from biogeme.expressions import Beta, log
from scipy.stats import chi2

In [2]:
df = pd.read_csv('swissmetro.dat', '\t')
database = db.Database('swissmetro', df)

# The following statement allows you to use the names of the
# variable as Python variable.
globals().update(database.variables)

# Removing some observations
exclude = CHOICE == 0
database.remove(exclude)

# Dummy variables variables for segmentation
age_00_24 = AGE == 1
age_25_39 = AGE == 2
age_40_54 = AGE == 3
age_55_65 = AGE == 4
age_65_plus = AGE == 5


female = 1 - MALE
male = MALE
noGA = GA == 0

FIRST_CLASS = FIRST
SECOND_CLASS = FIRST == 0

# Parameters to be estimated
ASC_CAR_MALE = Beta('ASC_CAR_MALE', 0, None, None, 0)
ASC_CAR_FEMALE = Beta('ASC_CAR_FEMALE', 0, None, None, 0)
ASC_CAR = ASC_CAR_MALE * male + ASC_CAR_FEMALE * female

ASC_TRAIN_MALE = Beta('ASC_TRAIN_MALE', 0, None, None, 0)
ASC_TRAIN_FEMALE = Beta('ASC_TRAIN_FEMALE', 0, None, None, 0)
ASC_TRAIN = ASC_TRAIN_MALE * male + ASC_TRAIN_FEMALE * female

B_TIME_CAR = Beta('B_TIME_CAR', 0, None, None, 0)

B_TIME_TRAIN_GA = Beta('B_TIME_TRAIN_GA', 0, None, None, 0)
B_TIME_TRAIN_noGA = Beta('B_TIME_TRAIN_noGA', 0, None, None, 0)
B_TIME_TRAIN = B_TIME_TRAIN_GA * GA + B_TIME_TRAIN_noGA * noGA

B_TIME_SM_GA = Beta('B_TIME_SM_GA', 0, None, None, 0)
B_TIME_SM_noGA = Beta('B_TIME_SM_noGA', 0, None, None, 0)
B_TIME_SM = B_TIME_SM_GA * GA + B_TIME_SM_noGA * noGA

B_COST_CAR_FIRST = Beta('B_COST_CAR_FIRST', 0, None, None, 0)
B_COST_CAR_SECOND = Beta('B_COST_CAR_SECOND', 0, None, None, 0)
B_COST_CAR = B_COST_CAR_FIRST * FIRST_CLASS + B_COST_CAR_SECOND * SECOND_CLASS

B_COST_TRAIN_FIRST = Beta('B_COST_TRAIN_FIRST', 0, None, None, 0)
B_COST_TRAIN_SECOND = Beta('B_COST_TRAIN_SECOND', 0, None, None, 0)
B_COST_TRAIN = B_COST_TRAIN_FIRST * FIRST_CLASS + B_COST_TRAIN_SECOND * SECOND_CLASS

B_COST_SM_FIRST = Beta('B_COST_SM_FIRST', 0, None, None, 0)
B_COST_SM_SECOND = Beta('B_COST_SM_SECOND', 0, None, None, 0)
B_COST_SM = B_COST_SM_FIRST * FIRST_CLASS + B_COST_SM_SECOND * SECOND_CLASS

B_HEADWAY_TRAIN_00_24 = Beta('B_HEADWAY_TRAIN_00_24', 0, None, None, 0)
B_HEADWAY_TRAIN_25_39 = Beta('B_HEADWAY_TRAIN_25_39', 0, None, None, 0)
B_HEADWAY_TRAIN_40_54 = Beta('B_HEADWAY_TRAIN_40_54', 0, None, None, 0)
B_HEADWAY_TRAIN_55_65 = Beta('B_HEADWAY_TRAIN_55_65', 0, None, None, 0)
B_HEADWAY_TRAIN_65_plus = Beta('B_HEADWAY_TRAIN_65_plus', 0, None, None, 0)

B_HEADWAY_TRAIN = B_HEADWAY_TRAIN_00_24 * age_00_24 + \
    B_HEADWAY_TRAIN_25_39 * age_25_39 + \
    B_HEADWAY_TRAIN_40_54 * age_40_54 + \
    B_HEADWAY_TRAIN_55_65 * age_55_65 + \
    B_HEADWAY_TRAIN_65_plus * age_65_plus

B_HEADWAY_SM_00_24 = Beta('B_HEADWAY_SM_00_24', 0, None, None, 0)
B_HEADWAY_SM_25_39 = Beta('B_HEADWAY_SM_25_39', 0, None, None, 0)
B_HEADWAY_SM_40_54 = Beta('B_HEADWAY_SM_40_54', 0, None, None, 0)
B_HEADWAY_SM_55_65 = Beta('B_HEADWAY_SM_55_65', 0, None, None, 0)
B_HEADWAY_SM_65_plus = Beta('B_HEADWAY_SM_65_plus', 0, None, None, 0)

B_HEADWAY_SM = B_HEADWAY_SM_00_24 * age_00_24 + \
    B_HEADWAY_SM_25_39 * age_25_39 + \
    B_HEADWAY_SM_40_54 * age_40_54 + \
    B_HEADWAY_SM_55_65 * age_55_65 + \
    B_HEADWAY_SM_65_plus * age_65_plus

# Definition of new variables
SM_COST = SM_CO * (GA == 0)
TRAIN_COST = TRAIN_CO * (GA == 0)
TRAIN_TT_SCALED = TRAIN_TT / 60
TRAIN_COST_SCALED = TRAIN_COST / 100
SM_TT_SCALED = SM_TT / 60
SM_COST_SCALED = SM_COST / 100
CAR_TT_SCALED = CAR_TT / 60
CAR_COST_SCALED = CAR_CO / 100

TRAIN_HE_SCALED = TRAIN_HE / 60
SM_HE_SCALED = SM_HE / 60

def piecewise_cost(x):
    """
    Piecewise linear transformation of the variable
    """
    piecewiseVariables = models.piecewiseVariables(x, [0, 0.5, 1, 1.75, None])
    return (piecewiseVariables[0] +
            Beta('pw_cost_0.5_1', 0, None, None, 0) * piecewiseVariables[1] +
            Beta('pw_cost_1_1.75', 0, None, None, 0) * piecewiseVariables[2] +
            Beta('pw_cost_1.75_more', 0, None, None, 0) * piecewiseVariables[3])

# Definition of the utility functions
V1 = ASC_TRAIN + \
     B_TIME_TRAIN * log(TRAIN_TT_SCALED) + \
     B_COST_TRAIN * piecewise_cost(TRAIN_COST_SCALED) + \
     B_HEADWAY_TRAIN * TRAIN_HE**0.5

V2 = B_TIME_SM * log(SM_TT_SCALED) + \
     B_COST_SM * piecewise_cost(SM_COST_SCALED) + \
     B_HEADWAY_SM * SM_HE**0.5

V3 = ASC_CAR + \
     B_TIME_CAR * log(CAR_TT_SCALED) + \
     B_COST_CAR * piecewise_cost(CAR_COST_SCALED)

# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3}

# Associate the availability conditions with the alternatives
av = {1: TRAIN_AV,
      2: SM_AV,
      3: CAR_AV}

## Estimation of a logit model

In [3]:
logprob = models.loglogit(V, av, CHOICE)
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'logit'
logit_results = biogeme.estimate(algorithm=opt.bioNewton)

In [4]:
print(logit_results.shortSummary())

Results for model logit
Nbr of parameters:		28
Sample size:			10719
Excluded data:			9
Final log likelihood:		-7645.798
Akaike Information Criterion:	15347.6
Bayesian Information Criterion:	15551.43



In [5]:
logit_results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR_FEMALE,-1.183261,0.145425,-8.136576,4.440892e-16,0.156263,-7.572249,3.663736e-14
ASC_CAR_MALE,-0.946221,0.138697,-6.822235,8.963497e-12,0.146515,-6.458196,1.059581e-10
ASC_TRAIN_FEMALE,1.422112,0.179676,7.914867,2.442491e-15,0.182275,7.802022,5.995204e-15
ASC_TRAIN_MALE,0.748675,0.174428,4.292169,1.769362e-05,0.177682,4.213581,2.513530e-05
B_COST_CAR_FIRST,-1.529316,0.223512,-6.842217,7.797762e-12,0.221499,-6.904403,5.041523e-12
B_COST_CAR_SECOND,-0.777908,0.157667,-4.933876,8.061382e-07,0.154275,-5.042335,4.598842e-07
B_COST_SM_FIRST,-2.214572,0.316253,-7.002543,2.513545e-12,0.335885,-6.593244,4.303180e-11
B_COST_SM_SECOND,-2.018703,0.273399,-7.383735,1.538769e-13,0.296649,-6.805026,1.010303e-11
B_COST_TRAIN_FIRST,-2.459433,0.342295,-7.185133,6.714629e-13,0.366979,-6.701838,2.058131e-11
B_COST_TRAIN_SECOND,-1.886225,0.269989,-6.986304,2.822187e-12,0.291228,-6.476790,9.369439e-11


## Nested logit

There are three possibilities to partition the choice set:

- [Car, Train]  and [Swissmetro],
- [Train, Swissmetro] and [Car],
- [Car, Swissmetro] and [Train].

The first one groups existing alternatives together. The second one groups public transportation modes together. The third one being less intuitive, we select the two first specifications. 


### Nested logit: existing alternatives

In [6]:
MU = Beta('MU', 1, 0, None, 0)
existing = MU, [1, 3]
future = 1.0, [2]
nests = existing, future
logprob = models.lognested(V, av, nests, CHOICE)
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'nested_existing'
nested_existing_results = biogeme.estimate(algorithm=opt.bioNewton)

In [7]:
print(nested_existing_results.shortSummary())

Results for model nested_existing
Nbr of parameters:		29
Sample size:			10719
Excluded data:			9
Final log likelihood:		-7640.153
Akaike Information Criterion:	15338.31
Bayesian Information Criterion:	15549.42



In [8]:
nested_existing_table = nested_existing_results.getEstimatedParameters()
nested_existing_table

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR_FEMALE,-0.990503,0.145141,-6.824397,8.829604e-12,0.156799,-6.317037,2.666265e-10
ASC_CAR_MALE,-0.796990,0.134874,-5.909160,3.438574e-09,0.142705,-5.584867,2.338786e-08
ASC_TRAIN_FEMALE,1.296364,0.175498,7.386771,1.505462e-13,0.179866,7.207388,5.704326e-13
ASC_TRAIN_MALE,0.703436,0.166884,4.215107,2.496595e-05,0.170429,4.127449,3.668101e-05
B_COST_CAR_FIRST,-1.321771,0.216669,-6.100429,1.057842e-09,0.220426,-5.996430,2.017028e-09
B_COST_CAR_SECOND,-0.722492,0.146314,-4.937944,7.895032e-07,0.143263,-5.043131,4.579747e-07
B_COST_SM_FIRST,-1.810215,0.301456,-6.004896,1.914550e-09,0.325937,-5.553871,2.794110e-08
B_COST_SM_SECOND,-1.693527,0.264345,-6.406503,1.488953e-10,0.286266,-5.915915,3.300350e-09
B_COST_TRAIN_FIRST,-1.900910,0.327453,-5.805141,6.431200e-09,0.359871,-5.282199,1.276422e-07
B_COST_TRAIN_SECOND,-1.484573,0.255399,-5.812754,6.145344e-09,0.278669,-5.327374,9.964310e-08


The nested parameter is greater than one, consistently with the theory.

In [9]:
mu = nested_existing_table.loc['MU','Value']
mu

1.1886366655405471

If we test the null hypothesis that the true value of MU is 1, we use a $t$-test:

In [10]:
mu_stderr = nested_existing_table.loc['MU', 'Rob. Std err']
tested_value = 1
ttest = (tested_value - mu) / mu_stderr
ttest

-2.7654135992446056

Therefore, we can reject the null hypothesis at the 5% level. It means that we reject logit.

We can also test the null hypothesis that the two models are equivalent using a likelihood ratio test: 

In [11]:
LL_logit = logit_results.data.logLike
LL_nested_existing = nested_existing_results.data.logLike
LR = -2 * (LL_logit - LL_nested_existing)
LR

11.290573204418251

Number of degrees of freedom:

In [12]:
dof = nested_existing_results.data.nparam - logit_results.data.nparam
dof

1

The threshold value of the $\chi$-square test with one degree of freedom at 5% level is:

In [13]:
chi2.isf(0.05, dof)

3.8414588206941285

Therefore, the null hypothesis can be rejected, and the nested logit model is preferred. 

### Nested logit: public transportation modes


In [14]:
MU = Beta('MU', 1, 0, None, 0)
public = MU, [1, 2]
private = 1.0, [3]
nests = public, private
logprob = models.lognested(V, av, nests, CHOICE)
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'nested_public'
nested_public_results = biogeme.estimate(algorithm=opt.bioNewton)

In [15]:
nested_public_table = nested_public_results.getEstimatedParameters()
nested_public_table

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR_FEMALE,-0.649372,0.199810,-3.249944,1.154276e-03,0.270657,-2.399245,1.642891e-02
ASC_CAR_MALE,-0.510585,0.178951,-2.853216,4.327922e-03,0.232788,-2.193350,2.828221e-02
ASC_TRAIN_FEMALE,2.275099,0.338732,6.716520,1.861156e-11,0.389803,5.836537,5.329696e-09
ASC_TRAIN_MALE,0.998379,0.287961,3.467068,5.261697e-04,0.296153,3.371157,7.485306e-04
B_COST_CAR_FIRST,-1.184524,0.239559,-4.944604,7.629863e-07,0.261182,-4.535250,5.753530e-06
B_COST_CAR_SECOND,-0.682967,0.158644,-4.305020,1.669709e-05,0.156222,-4.371774,1.232412e-05
B_COST_SM_FIRST,-1.583973,0.326648,-4.849177,1.239748e-06,0.370116,-4.279663,1.871767e-05
B_COST_SM_SECOND,-1.540185,0.295412,-5.213683,1.851278e-07,0.323164,-4.765950,1.879662e-06
B_COST_TRAIN_FIRST,-1.880249,0.392789,-4.786917,1.693628e-06,0.485064,-3.876287,1.060624e-04
B_COST_TRAIN_SECOND,-1.264360,0.328550,-3.848299,1.189410e-04,0.453511,-2.787938,5.304478e-03


The nest parameter is less than 1. This is inconsistent with the theory. The model is rejected.

In [16]:
nested_public_table.loc['MU','Value']

0.49625298277552676

In conclusion, among the three models, the nested logit model where the existing alternatives are in the same nest is preferred. 